In [1]:
import os
import gdown
import tarfile
import zipfile
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

ModuleNotFoundError: No module named 'gdown'

In [3]:
# List of anime character names
anime_characters = ['amelia_watson', 'Anya_Forger', 'Aquamarine_Hoshino', 'arima_kousei', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 
                   'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'fern', 'frieren', 'Ganyu_genshin', 'gawr_gura', 'Gojo_Satoru', 
                   'hoshino_ai', 'hutao_genshin', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'kaori', 'Keqing_genshin', 'Killjoy_Valorant', 
                   'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'mikasa', 'Minato_Aqua', 
                   'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact'
                   , 'Ruby_Hoshino', 'Sakura_Haruno', 'tanjiro', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 
                   'violet_evergarden', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'zeta_hololive']

# Count the number of characters
num_characters = len(anime_characters)

# Print the result
print("Number of anime characters:", num_characters)


Number of anime characters: 46


In [3]:
file_id = '15M-lNTPksPL3RusWIaM0wsgNTHhrPxif'
file_url = f'https://drive.google.com/uc?id={file_id}'

In [4]:
destination_dir = '/content/images'
os.makedirs(destination_dir, exist_ok=True)

In [6]:
# Download the file
zip_file_path = os.path.join(destination_dir, 'images.zip')
gdown.download(file_url, zip_file_path, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=15M-lNTPksPL3RusWIaM0wsgNTHhrPxif
To: /content/images/images.zip
100%|██████████| 258M/258M [00:04<00:00, 58.5MB/s]


'/content/images/images.zip'

In [7]:
# Extract the file based on its extension
if zip_file_path.endswith('.zip'):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_dir)
elif zip_file_path.endswith(('.tar', '.tar.gz', '.tgz')):
    with tarfile.open(zip_file_path, 'r') as tar_ref:
        tar_ref.extractall(destination_dir)
else:
    print(f"Unsupported file extension: {zip_file_path}")

In [8]:
# Define the dataset directory
dataset_dir = '/content/images/datasets_character_anime'  # Update if needed

In [9]:
# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Set the validation split if needed
)

# Create a generator for the training set
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Create a generator for the validation set
validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 4043 images belonging to 43 classes.
Found 1002 images belonging to 43 classes.


In [10]:
# Display class names
class_names = list(train_generator.class_indices.keys())
print("Class names:", class_names)

Class names: ['Anya_Forger', 'Aquamarine_Hoshino', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'Ganyu_genshin', 'Gojo_Satoru', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'Keqing_genshin', 'Killjoy_Valorant', 'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'Minato_Aqua', 'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact', 'Ruby_Hoshino', 'Sakura_Haruno', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'amelia_watson', 'arima_kousei', 'gawr_gura', 'hoshino_ai', 'hutao_genshin', 'mikasa', 'tanjiro', 'violet_evergarden', 'zeta_hololive']


In [11]:
# Model
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(Conv2D(128, (3, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2)))
cnn.add(BatchNormalization())

cnn.add(Flatten())
cnn.add(Dense(512, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(43, activation='softmax'))

In [49]:
cnn.add(Dense(43, activation='softmax'))  # Assuming you have 43 classes


In [12]:
# Compile the model
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Print model summary
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 111, 111, 32)      128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                        

In [14]:
history = cnn.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

# Save the trained model
model_save_path = '/content/your_model.h5'  # Replace with the desired path and filename
cnn.save(model_save_path)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


127/127 [==============================] - 364s 3s/step - loss: 15.6860 - accuracy: 0.0492 - val_loss: 11.7040 - val_accuracy: 0.0319
Epoch 2/10
127/127 [==============================] - 376s 3s/step - loss: 6.7763 - accuracy: 0.0524 - val_loss: 6.9249 - val_accuracy: 0.0329
Epoch 3/10
127/127 [==============================] - 368s 3s/step - loss: 4.4933 - accuracy: 0.0576 - val_loss: 6.2081 - val_accuracy: 0.0489
Epoch 4/10
127/127 [==============================] - 363s 3s/step - loss: 3.8946 - accuracy: 0.0653 - val_loss: 3.7567 - val_accuracy: 0.0479
Epoch 5/10
127/127 [==============================] - 371s 3s/step - loss: 3.7725 - accuracy: 0.0663 - val_loss: 3.7504 - val_accuracy: 0.0539
Epoch 6/10
127/127 [==============================] - 369s 3s/step - loss: 3.7317 - accuracy: 0.0732 - val_loss: 3.8574 - val_accuracy: 0.0649
Epoch 7/10
127/127 [==============================] - 376s 3s/step - loss: 3.6820 - accuracy: 0.0787 - val_loss: 3.6600 - val_accuracy: 0.0749
Epoch 8/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

# Load the trained model
model_path = 'new_anime_classification.h5'  # Replace with the actual path to your model
loaded_model = load_model(model_path)

# Function to preprocess the input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0

# Function to make predictions
def predict_image(model, img_path):
    processed_image = preprocess_image(img_path)
    prediction = model.predict(processed_image)
    return prediction

# Replace 'your_image.jpg' with the actual path to the image you want to test
image_path = 'tanjiro.png'
result = predict_image(loaded_model, image_path)
# Assuming you have 36 classes, replace this with your actual class names
class_names = ['amelia_watson', 'Anya_Forger', 'Aquamarine_Hoshino', 'arima_kousei', 'Ayaka_Genshin_Impact', 'Boa_Hancock', 'Charlotte_Genshin_Impact', 'Damian_Desmond', 'Dazai_Osamu_BSD', 'Ganyu_genshin', 'gawr_gura', 'Gojo_Satoru', 'hoshino_ai', 'hutao_genshin', 'Jett_Valorant', 'Kafka_Honkai_Star_Rail', 'Keqing_genshin', 'Killjoy_Valorant', 'Kobo_kanaeru', 'Kugisaki_Nobara', 'Loid_Forger', 'Luffy_D_Monkey', 'Midoriya_Izuku', 'mikasa', 'Minato_Aqua', 'Misa_Amane', 'Mitsuri_Kanroji', 'Nico_Robin', 'Nier_Automata_9S', 'Nier_Automata_A2', 'Raiden_Shogun_Genshin_Impact', 'Ruby_Hoshino', 'Sakura_Haruno', 'tanjiro', 'Todoroki_Shoto', 'Tokisaki_Kurumi', 'Uraraka_Ochako', 'violet_evergarden', 'Wanderer', 'Yor_Forger', 'Yuri_Briar', 'Zerotwo', 'zeta_hololive']

# Get the predicted class index
predicted_class_index = np.argmax(result)
predicted_class = class_names[predicted_class_index]

print(f"The predicted class is: {predicted_class}")

1/1 [==============================] - 0s 265ms/step


IndexError: list index out of range